In [ ]:
import sys
from pathlib import Path

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

IN_DIR = DIR/'example_data'/'inputs'
OUT_DIR = DIR/'example_data'/'outputs'

%load_ext autoreload
%autoreload 2



In [ ]:
# # Download test elevation data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_srtm_tile_names(lon_lats)
# print('Tiles to download =', tiles)
    
# wt.download_topography_nasa(tiles, TEST_DATA_DIR)

In [3]:
transmitters = wt.read_transmitters(IN_DIR/'transmitters.csv')[:2]
high_definition = False


In [4]:
# Download elevation data
lon_lats = wt.get_lon_lats(transmitters)
tiles = wt.get_srtm_tile_names(lon_lats)
print('Tiles to download =', tiles)

username = None
password = None
if high_definition:
    path = IN_DIR/'elevations-hd'
    try:
        username = wt.get_secret('NASA_USERNAME')
        password = wt.get_secret('NASA_PASSWORD')
    except:
        pass
else:
    path = IN_DIR/'elevations-sd'
    
wt.download_topography_nasa(tiles, path, high_definition=high_definition,
  username=username, password=password)

Tiles to download = ['S36E174', 'S35E173']


In [7]:
# Create SPLAT transmitter files

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

CanUCMe_21.az     MyNetwork_11.qth  MyNetwork_9.lrp   Skynet_14.el
CanUCMe_21.el     MyNetwork_5.az    MyNetwork_9.qth   Skynet_14.lrp
CanUCMe_21.lrp    MyNetwork_5.el    RuralCom_18.az    Skynet_14.qth
CanUCMe_21.qth    MyNetwork_5.lrp   RuralCom_18.el    Skynet_15.az
CanUCMe_22.az     MyNetwork_5.qth   RuralCom_18.lrp   Skynet_15.el
CanUCMe_22.el     MyNetwork_6.az    RuralCom_18.qth   Skynet_15.lrp
CanUCMe_22.lrp    MyNetwork_6.el    RuralCom_19.az    Skynet_15.qth
CanUCMe_22.qth    MyNetwork_6.lrp   RuralCom_19.el    Skynet_16.az
CanUCMe_23.az     MyNetwork_6.qth   RuralCom_19.lrp   Skynet_16.el
CanUCMe_23.el     MyNetwork_7.az    RuralCom_19.qth   Skynet_16.lrp
CanUCMe_23.lrp    MyNetwork_7.el    Skynet_12.az      Skynet_16.qth
CanUCMe_23.qth    MyNetwork_7.lrp   Skynet_12.el      Skynet_17.az
MyNetwork_10.az   MyNetwork_7.qth   Skynet_12.lrp     Skynet_17.el
MyNetwork_10.el   MyNetwork_8.az    Skynet_12.qth     Skynet_17.lrp
MyNetwork_10.lrp  MyNetwork_8.el    Skynet_13.az      S

In [8]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'elevations-hd'
else:
    in_path = IN_DIR/'elevations-sd'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

-35:-34:186:187.sdf  MyNetwork_6.az       Skynet_12.lrp
-36:-35:185:186.sdf  MyNetwork_6.el       Skynet_12.qth
CanUCMe_21.az        MyNetwork_6.lrp      Skynet_13.az
CanUCMe_21.el        MyNetwork_6.qth      Skynet_13.el
CanUCMe_21.lrp       MyNetwork_7.az       Skynet_13.lrp
CanUCMe_21.qth       MyNetwork_7.el       Skynet_13.qth
CanUCMe_22.az        MyNetwork_7.lrp      Skynet_14.az
CanUCMe_22.el        MyNetwork_7.qth      Skynet_14.el
CanUCMe_22.lrp       MyNetwork_8.az       Skynet_14.lrp
CanUCMe_22.qth       MyNetwork_8.el       Skynet_14.qth
CanUCMe_23.az        MyNetwork_8.lrp      Skynet_15.az
CanUCMe_23.el        MyNetwork_8.qth      Skynet_15.el
CanUCMe_23.lrp       MyNetwork_9.az       Skynet_15.lrp
CanUCMe_23.qth       MyNetwork_9.el       Skynet_15.qth
MyNetwork_10.az      MyNetwork_9.lrp      Skynet_16.az
MyNetwork_10.el      MyNetwork_9.qth      Skynet_16.el
MyNetwork_10.lrp     RuralCom_18.az       Skynet_16.lrp
MyNetwork_10.qth     RuralCom_18.el       Skynet_16.qth


In [10]:
# Create coverage reports

transmitter_names = [t['name'] for t in transmitters[:1]]
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)

% ls {out_path}

Timing create_coverage_reports...
2016-08-10 13:25:14.990288   Began process
2016-08-10 13:26:24.199369   Finished in 1.15 min
MyNetwork_5-ck.ppm           MyNetwork_5.kml
MyNetwork_5-site_report.txt  MyNetwork_5.ppm


In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path, delete_ppm=False)

% ls {path}

In [ ]:
import re
import subprocess


re.search(r"<west>([0-9-][0-9\.]*)<\/west>", '').group(1)